In [1]:
import fjos
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import joblib
from sklearn.ensemble import GradientBoostingRegressor
import seaborn as sns

In [19]:
# mst = [input('Iveskite stoties pav. (pvz. vilniaus-ams)')]
# prad = input('iveskite diapazono pradzia (yy-mm-dd)')
# pab = input('iveskite diapazono pabaiga (yy-mm-dd)')

prad = '2022-07-01'
pab = '2024-07-01'
mst = ['vilniaus-ams']

# fjos.data_extr_meteo(mst, prad, pab)

# mst_pd = fjos.file_to_pd(f'{' '.join(mst)}.csv')
litgrid_raw_pd = fjos.litgrid_data_to_pd('C:\\Users\\runda\\Desktop\\Python mokymai\\Galutinis Projektas Elektros prognoze vs Orai\\Duomenys\\dashboard_data_2022-07-01_2024-07-01.csv')
litgrid_pd = litgrid_raw_pd[(litgrid_raw_pd['Time']>= prad) & (litgrid_raw_pd['Time']<= f'{pab} 23:00:00')]
miestas = fjos.meteo_file_to_pd(f'C:\\Users\\runda\\Desktop\\Python mokymai\\Galutinis Projektas Elektros prognoze vs Orai\\Duomenys\\{' '.join(mst)}.csv')

In [20]:
df = pd.concat([litgrid_pd.reset_index(), miestas], axis="columns")
df = df.drop(columns=['Time(Meteo)', 'index'])
df = df.set_index('Time')


In [21]:
litgrid_raw_pd

,Time,Prognozuojamas nacionalinis elektros energijos suvartojimas,Planuojamas nacionalinis elektros energijos suvartojimas,Faktinis nacionalinis Elektros energijos vartojimas,Month,Week,Hour
0,2022-07-01 00:00:00,1225.0,1160.57,1246.044,7,4,0
1,2022-07-01 01:00:00,1143.0,1078.77,1163.384,7,4,1
2,2022-07-01 02:00:00,1114.0,1041.07,1130.666,7,4,2
3,2022-07-01 03:00:00,1100.0,1035.27,1118.605,7,4,3
4,2022-07-01 04:00:00,1073.0,1021.67,1112.705,7,4,4
...,...,...,...,...,...,...,...
17563,2024-07-01 19:00:00,1428.0,1357.37,1429.136,7,0,19
17564,2024-07-01 20:00:00,1411.0,1376.87,1363.344,7,0,20
17565,2024-07-01 21:00:00,1382.0,1316.97,1348.061,7,0,21
17566,2024-07-01 22:00:00,1329.0,1295.97,1321.252,7,0,22


In [22]:
litgrid_pd.head(30)

,Time,Prognozuojamas nacionalinis elektros energijos suvartojimas,Planuojamas nacionalinis elektros energijos suvartojimas,Faktinis nacionalinis Elektros energijos vartojimas,Month,Week,Hour
0,2022-07-01 00:00:00,1225.0,1160.57,1246.044,7,4,0
1,2022-07-01 01:00:00,1143.0,1078.77,1163.384,7,4,1
2,2022-07-01 02:00:00,1114.0,1041.07,1130.666,7,4,2
3,2022-07-01 03:00:00,1100.0,1035.27,1118.605,7,4,3
4,2022-07-01 04:00:00,1073.0,1021.67,1112.705,7,4,4
5,2022-07-01 05:00:00,1117.0,1042.47,1141.764,7,4,5
6,2022-07-01 06:00:00,1289.0,1244.57,1319.119,7,4,6
7,2022-07-01 07:00:00,1471.0,1390.17,1500.649,7,4,7
8,2022-07-01 08:00:00,1604.0,1518.07,1647.762,7,4,8
9,2022-07-01 09:00:00,1660.0,1545.77,1734.751,7,4,9


In [23]:
Xtest = df[['Month', 'Week', 'Hour', 'airTemperature', 'cloudCover']]

In [24]:
modelis = joblib.load('best_pipeline_gradient.pkl')
# modelis = joblib.load('best_pipeline_tree.pkl')
# modelis = joblib.load('best_pipeline_ridge_poly.pkl')

In [25]:
pred= modelis.predict(Xtest)
spejimai = pd.Series(data=pred, name='Spėjimas')
es_test = pd.concat([df.reset_index(), spejimai], axis=1)
es_test.set_index('Laikas', inplace=True)


gradientr2 = r2_score(es_test['Faktinis nacionalinis Elektros energijos vartojimas'], es_test['Spėjimas'])

gradientr2

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Hour
- Month
- Week
Feature names seen at fit time, yet now missing:
- Savaite
- Valanda
- menuo


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[15,10])
sns.lineplot(es_test[['Faktinis nacionalinis Elektros energijos vartojimas', 'Prognozuojamas nacionalinis elektros energijos suvartojimas', 'Spėjimas']])